In [1]:
{-# LANGUAGE NegativeLiterals #-}
{-# LANGUAGE FlexibleContexts #-}

In [113]:
import Text.Parsec 
import Text.ParserCombinators.Parsec.Number
import Data.List (partition, intersect, sortBy, groupBy, sort, group, (\\))
import qualified Data.Set as S
import Data.Function (on)

In [151]:
import Debug.Trace

In [3]:
data Programx = Programx String Int [String]
                deriving (Show, Eq)

name (Programx n _ _) = n 
weight (Programx _ w _) = w
supports (Programx _ _ s) = s

In [4]:
data Treex = Treex Programx [Treex] Int deriving (Show, Eq)
root (Treex p _ _) = p
trees (Treex _ t _) = t
tWeight (Treex _ _ w) = w

In [5]:
onlySpaces = many (oneOf " \t")
parens = between (string "(") (string ")")
sym = many lower
commaSep sym = sym `sepBy` (onlySpaces *> string "," *> onlySpaces)

In [6]:
mFile = mLine `sepBy` newline 
mLine = Programx <$> sym <*> (onlySpaces *> (parens int)) <*> supportsP
supportsP = (onlySpaces *> (string "->") *> onlySpaces *> (commaSep sym)) <|> (pure [])

In [7]:
parseFile :: String -> Either ParseError [Programx]
parseFile input = parse mFile "(unknown)" input

parseLine :: String -> Either ParseError Programx
parseLine input = parse mLine "(unknown)" input

successfulParse :: Either ParseError [a] -> [a]
successfulParse (Left _) = []
successfulParse (Right a) = a

In [8]:
parseLine "kuvqhnm (77)"

Right (Programx "kuvqhnm" 77 [])

In [9]:
parseLine "dihjv (2158) -> gausx, ncdmp, hozgrub"

Right (Programx "dihjv" 2158 ["gausx","ncdmp","hozgrub"])

In [10]:
sample = "pbga (66)\nxhth (57)\nebii (61)\nhavc (66)\nktlj (57)\nfwft (72) -> ktlj, cntj, xhth\nqoyq (66)\npadx (45) -> pbga, havc, qoyq\ntknk (41) -> ugml, padx, fwft\njptl (61)\nugml (68) -> gyxo, ebii, jptl\ngyxo (61)\ncntj (57)"

In [11]:
-- sample = "pbga (66)\nxhth (57)"

In [12]:
print sample

"pbga (66)\nxhth (57)\nebii (61)\nhavc (66)\nktlj (57)\nfwft (72) -> ktlj, cntj, xhth\nqoyq (66)\npadx (45) -> pbga, havc, qoyq\ntknk (41) -> ugml, padx, fwft\njptl (61)\nugml (68) -> gyxo, ebii, jptl\ngyxo (61)\ncntj (57)"

In [13]:
successfulParse $ parseFile sample

[Programx "pbga" 66 [],Programx "xhth" 57 [],Programx "ebii" 61 [],Programx "havc" 66 [],Programx "ktlj" 57 [],Programx "fwft" 72 ["ktlj","cntj","xhth"],Programx "qoyq" 66 [],Programx "padx" 45 ["pbga","havc","qoyq"],Programx "tknk" 41 ["ugml","padx","fwft"],Programx "jptl" 61 [],Programx "ugml" 68 ["gyxo","ebii","jptl"],Programx "gyxo" 61 [],Programx "cntj" 57 []]

In [14]:
programs :: [Programx] -> S.Set String
programs = S.fromList . map name

In [15]:
pr = programs $ successfulParse $ parseFile sample
pr

fromList ["cntj","ebii","fwft","gyxo","havc","jptl","ktlj","padx","pbga","qoyq","tknk","ugml","xhth"]

In [16]:
supported :: [Programx] -> S.Set String
supported = S.unions . map (S.fromList . supports)

In [17]:
su = supported $ successfulParse $ parseFile sample
su

fromList ["cntj","ebii","fwft","gyxo","havc","jptl","ktlj","padx","pbga","qoyq","ugml","xhth"]

In [18]:
print $ head $ S.elems $ S.difference pr su

"tknk"

In [19]:
part1 :: [Programx] -> String
part1 progs = head $ S.elems $ S.difference pr su
    where su = supported progs
          pr = programs progs

In [20]:
main :: IO ()
main = do 
        text <- readFile "../../data/advent07.txt"
        let progs = successfulParse $ parseFile text
        print $ part1 progs

In [21]:
main

"vtzay"

In [28]:
makeSingletons :: [Programx] -> ([Treex], [Programx])
makeSingletons programs = (trees, others)
    where (sPrograms, others) = partition isLeaf programs
          isLeaf pr = null $ supports pr
          trees = map makeSTree sPrograms
          makeSTree pr = Treex pr [] (weight pr)

In [29]:
makeSingletons $ successfulParse $ parseFile sample

([Treex (Programx "pbga" 66 []) [] 66,Treex (Programx "xhth" 57 []) [] 57,Treex (Programx "ebii" 61 []) [] 61,Treex (Programx "havc" 66 []) [] 66,Treex (Programx "ktlj" 57 []) [] 57,Treex (Programx "qoyq" 66 []) [] 66,Treex (Programx "jptl" 61 []) [] 61,Treex (Programx "gyxo" 61 []) [] 61,Treex (Programx "cntj" 57 []) [] 57],[Programx "fwft" 72 ["ktlj","cntj","xhth"],Programx "padx" 45 ["pbga","havc","qoyq"],Programx "tknk" 41 ["ugml","padx","fwft"],Programx "ugml" 68 ["gyxo","ebii","jptl"]])

In [42]:
makeTree :: [Treex] -> Programx -> Treex
makeTree trees program = Treex program subtrees (w + (weight program))
    where subtrees = filter (\t -> (name $ root t) `elem` (supports program)) trees
          w = sum $ map tWeight subtrees

In [43]:
addTreeLayer :: [Treex] -> [Programx] -> ([Treex], [Programx])
addTreeLayer trees programs = (trees', programs')
    where (sPrograms, others) = partition isSupporter programs
          isSupporter pr = not $ null $ (supports pr) `intersect` roots
          roots = map (name . root) trees
          trees' = map (makeTree trees) sPrograms
          newRoots = map root trees'
          programs' = programs \\ newRoots


In [44]:
(leaves, others) =  makeSingletons $ successfulParse $ parseFile sample
(leaves, others)

([Treex (Programx "pbga" 66 []) [] 66,Treex (Programx "xhth" 57 []) [] 57,Treex (Programx "ebii" 61 []) [] 61,Treex (Programx "havc" 66 []) [] 66,Treex (Programx "ktlj" 57 []) [] 57,Treex (Programx "qoyq" 66 []) [] 66,Treex (Programx "jptl" 61 []) [] 61,Treex (Programx "gyxo" 61 []) [] 61,Treex (Programx "cntj" 57 []) [] 57],[Programx "fwft" 72 ["ktlj","cntj","xhth"],Programx "padx" 45 ["pbga","havc","qoyq"],Programx "tknk" 41 ["ugml","padx","fwft"],Programx "ugml" 68 ["gyxo","ebii","jptl"]])

In [45]:
(trs, oths) = addTreeLayer leaves others
(trs, oths)

([Treex (Programx "fwft" 72 ["ktlj","cntj","xhth"]) [Treex (Programx "xhth" 57 []) [] 57,Treex (Programx "ktlj" 57 []) [] 57,Treex (Programx "cntj" 57 []) [] 57] 243,Treex (Programx "padx" 45 ["pbga","havc","qoyq"]) [Treex (Programx "pbga" 66 []) [] 66,Treex (Programx "havc" 66 []) [] 66,Treex (Programx "qoyq" 66 []) [] 66] 243,Treex (Programx "ugml" 68 ["gyxo","ebii","jptl"]) [Treex (Programx "ebii" 61 []) [] 61,Treex (Programx "jptl" 61 []) [] 61,Treex (Programx "gyxo" 61 []) [] 61] 251],[Programx "tknk" 41 ["ugml","padx","fwft"]])

In [46]:
length trs

3

In [47]:
length oths

1

In [52]:
(trs', oths') =  addTreeLayer trs oths
(trs', oths')

([Treex (Programx "tknk" 41 ["ugml","padx","fwft"]) [Treex (Programx "fwft" 72 ["ktlj","cntj","xhth"]) [Treex (Programx "xhth" 57 []) [] 57,Treex (Programx "ktlj" 57 []) [] 57,Treex (Programx "cntj" 57 []) [] 57] 243,Treex (Programx "padx" 45 ["pbga","havc","qoyq"]) [Treex (Programx "pbga" 66 []) [] 66,Treex (Programx "havc" 66 []) [] 66,Treex (Programx "qoyq" 66 []) [] 66] 243,Treex (Programx "ugml" 68 ["gyxo","ebii","jptl"]) [Treex (Programx "ebii" 61 []) [] 61,Treex (Programx "jptl" 61 []) [] 61,Treex (Programx "gyxo" 61 []) [] 61] 251] 778],[])

In [49]:
map tWeight trs

[243,243,251]

In [173]:
balancedTree :: Treex -> Bool
balancedTree tr 
    | null $ trees tr = True
    | otherwise = (1==) $ S.size $ S.fromList $ map tWeight $ trees tr

In [174]:
map balancedTree leaves

[True,True,True,True,True,True,True,True,True]

In [175]:
map balancedTree trs

[True,True,True]

In [176]:
map balancedTree trs'

[False]

In [177]:
treesByWeight :: Treex -> [Treex]
treesByWeight = sortBy (compare `on` tWeight) . trees

In [178]:
map tWeight $ treesByWeight $ head trs'

[243,243,251]

In [179]:
oddWeight :: Treex -> Int
oddWeight = tWeight . head . head .filter (\g -> length g == 1) . groupBy ((==) `on` tWeight) . trees

In [180]:
oddWeight :: Treex -> Int
oddWeight = head . head .filter (\g -> length g == 1) . group . sort . map tWeight . trees

In [181]:
oddWeight $ head trs'

251

In [182]:
-- oddMajorityWeight :: Treex -> (Int, Int)
oddMajorityWeight = extractWeights . oddMajority . groups
    where groups = group . sort . map tWeight . trees
          oddMajority = partition (\g -> length g == 1)
          extractWeights (o, m) = (head $ head o, head $ head m)

In [183]:
oddMajorityWeight $ head trs'

(251,243)

In [184]:
weight $ root $ head $ filter (\t -> tWeight t == 251) $ trees $ head trs' 

68

In [185]:
68 - 251 + 243

60

In [202]:
checkTrees :: ([Treex], [Programx]) -> Int
checkTrees (partTrees, programs) =
    if all balancedTree partTrees
    then trace (show $ length partTrees) checkTrees $ addTreeLayer partTrees programs
    else 
        oddTreeWeight - oddWeight + majorityWeight
    where 
        unbalancedTree = head $ filter (not . balancedTree) partTrees
        (oddWeight, majorityWeight) = oddMajorityWeight unbalancedTree
        unbalancedSubtrees = trees unbalancedTree
        oddTreeWeight = weight $ root $ head $ filter (\t -> tWeight t == oddWeight) $ trees unbalancedTree

In [188]:
ubt = head $ filter (not . balancedTree) trs'
ubt

Treex (Programx "tknk" 41 ["ugml","padx","fwft"]) [Treex (Programx "fwft" 72 ["ktlj","cntj","xhth"]) [Treex (Programx "xhth" 57 []) [] 57,Treex (Programx "ktlj" 57 []) [] 57,Treex (Programx "cntj" 57 []) [] 57] 243,Treex (Programx "padx" 45 ["pbga","havc","qoyq"]) [Treex (Programx "pbga" 66 []) [] 66,Treex (Programx "havc" 66 []) [] 66,Treex (Programx "qoyq" 66 []) [] 66] 243,Treex (Programx "ugml" 68 ["gyxo","ebii","jptl"]) [Treex (Programx "ebii" 61 []) [] 61,Treex (Programx "jptl" 61 []) [] 61,Treex (Programx "gyxo" 61 []) [] 61] 251] 778

In [189]:
trees ubt

[Treex (Programx "fwft" 72 ["ktlj","cntj","xhth"]) [Treex (Programx "xhth" 57 []) [] 57,Treex (Programx "ktlj" 57 []) [] 57,Treex (Programx "cntj" 57 []) [] 57] 243,Treex (Programx "padx" 45 ["pbga","havc","qoyq"]) [Treex (Programx "pbga" 66 []) [] 66,Treex (Programx "havc" 66 []) [] 66,Treex (Programx "qoyq" 66 []) [] 66] 243,Treex (Programx "ugml" 68 ["gyxo","ebii","jptl"]) [Treex (Programx "ebii" 61 []) [] 61,Treex (Programx "jptl" 61 []) [] 61,Treex (Programx "gyxo" 61 []) [] 61] 251]

In [238]:
sglPrs =  makeSingletons $ successfulParse $ parseFile sample
sglPrs

([Treex (Programx "pbga" 66 []) [] 66,Treex (Programx "xhth" 57 []) [] 57,Treex (Programx "ebii" 61 []) [] 61,Treex (Programx "havc" 66 []) [] 66,Treex (Programx "ktlj" 57 []) [] 57,Treex (Programx "qoyq" 66 []) [] 66,Treex (Programx "jptl" 61 []) [] 61,Treex (Programx "gyxo" 61 []) [] 61,Treex (Programx "cntj" 57 []) [] 57],[Programx "fwft" 72 ["ktlj","cntj","xhth"],Programx "padx" 45 ["pbga","havc","qoyq"],Programx "tknk" 41 ["ugml","padx","fwft"],Programx "ugml" 68 ["gyxo","ebii","jptl"]])

In [239]:
fst sglPrs

[Treex (Programx "pbga" 66 []) [] 66,Treex (Programx "xhth" 57 []) [] 57,Treex (Programx "ebii" 61 []) [] 61,Treex (Programx "havc" 66 []) [] 66,Treex (Programx "ktlj" 57 []) [] 57,Treex (Programx "qoyq" 66 []) [] 66,Treex (Programx "jptl" 61 []) [] 61,Treex (Programx "gyxo" 61 []) [] 61,Treex (Programx "cntj" 57 []) [] 57]

In [240]:
checkTrees sglPrs

60

In [242]:
all balancedTree leaves

True

In [243]:
leaves

[Treex (Programx "pbga" 66 []) [] 66,Treex (Programx "xhth" 57 []) [] 57,Treex (Programx "ebii" 61 []) [] 61,Treex (Programx "havc" 66 []) [] 66,Treex (Programx "ktlj" 57 []) [] 57,Treex (Programx "qoyq" 66 []) [] 66,Treex (Programx "jptl" 61 []) [] 61,Treex (Programx "gyxo" 61 []) [] 61,Treex (Programx "cntj" 57 []) [] 57]

In [244]:
balancedTree $ head leaves

True

In [245]:
head leaves

Treex (Programx "pbga" 66 []) [] 66

In [246]:
-- part2 :: [Programx] -> Int
part2 = checkTrees . makeSingletons 
-- part2 prs = all balancedTree $ fst $ addTreeLayer ts os
--     where (ts, os) = makeSingletons prs

In [247]:
main :: IO ()
main = do 
        text <- readFile "../../data/advent07.txt"
        let progs = successfulParse $ parseFile text
--         print $ part1 progs
        print $ part2 progs

In [248]:
main

Prelude.head: empty list